# Machine Learning in Python

## HSE, 2023-24

### Home Assignment #1. PCA

Assignment completed by:

    (insert your last name and first name)

### General Information

__Publication date:__ 16.05.2024

__Deadline:__ 04:00 24.05.2024

### Grading and penalties

The number of points for each problem in this homework assignment is listed next to the problem condition.

The homework grade is calculated using the following formula:

$$
s \times 10/53 ,
$$

where $s$ is the number of points you have scored in total for all tasks.

For submitting an assignment after the deadline, a penalty of 1 **secondary** point per day is applied to the final grade for the assignment, but the delay cannot exceed one week.

__WARNING!__ Homework is done independently. “Similar” solutions are considered plagiarism and all students involved (including those who have been copied from) cannot get more than 0 points for it.

Also, don't forget that all solutions are run through a special new anti-plagiarism system for Jupyter notebooks, which detects cross “similarities” between different notebooks, as well as solutions generated by a neural network. Such works will also mandatorily be regarded as plagiarism.

### Submission format

You upload your solution on the HA to [Anytask](https://anytask.org/) system. You need to upload a file with the extension .ipynb (Python notebook)

### About the assignment

In this assignment, we will practice working with linear algebra and, specifically, with the PCA algorithm. The goal of the assignment is to try to compress the soundtrack of a Beethoven melody using this dimensionality reduction algorithm, which we have discussed in detail in seminars and lectures.

First, let's import all the necessary libraries:

In [ ]:
# Needed to read and write audio files
from scipy.io import wavfile

# This is to play audio files directly in Notebook
from IPython.display import Audio

# And this is the standard set of data analysis libraries required for this assignment
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import ipywidgets as widgets
import time
from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.ndimage import gaussian_filter1d
from ipywidgets import HBox, VBox

Let's download the data:

Let's read the audio track using wavfile:


In [ ]:
samplerate, data = wavfile.read('Beethoven_Violin_Sonata_Op_96_first_movement_bars_1-22.wav')

In the cell above we see a certain `samplerate`. This variable stores the samplerate by default; the standard value for audio is 44100 Hz.

For those who are not very familiar with sound encoding and recording methods (aka those who didn't pass the 'EGE' in computer science at school :) ), - samplerate shows how many consecutive elements of the array with the signal encode a sound of 1 second duration.

You can learn more about sound encoding [here](https://ru.wikipedia.org/wiki/Кодирование_звуковой_информации).

So, let's see what the sampling rate of our audio track is.

In [ ]:
samplerate

If you divide the length of the signal array by `samplerate`, you will get the duration of the audio track in seconds. At least, you should get this result!

In [ ]:
len(data) / samplerate

Forty-five seconds. Does that sound about right? Compare it to the length of the track by running it on your computer.

Note also that the sound is stereo, as the signal is encoded with two channels (for the left and right speaker).

In [ ]:
data.shape

Draw the signals in both channels:

In [ ]:
# Channel 1
plt.figure(figsize=(20,5))
plt.plot(data[:,0])
plt.show()

# Channel 2
plt.figure(figsize=(20,5))
plt.plot(data[:,1])
plt.show()

Here it is - the legendary “cardiogram” sound we are all familiar with! :)

So now let's average the channels and get a mono sound, which will be much easier to work with.


In [ ]:
mono_sound = np.mean(data, axis=1)
mono_sound.shape

And now we're finally ready to hear what we have to compress! :)

In [ ]:
Audio(mono_sound, rate = samplerate)

Pretty, isn't it? :)

For convenience, we will also trim the signal array so that it can be easily divided into equal parts. These parts will form the dataset that you need to compress using the methods you know.

In fact, the method is very similar to the one we used to compress the picture at the seminar by splitting it into rectangular sub-pictures - only here the task is even simpler :)

In [ ]:
mono_sound_to_cut = mono_sound[:1990000]

Let's check that our sound is now just a vector of numbers:

In [ ]:
mono_sound_to_cut.shape

Great, everything is absolutely ready!

Well, that's the end of our mission; now it's your turn! :)

### 1


#### 1.1. (2 points)
Write a function that will split the signal into equal parts of length 1000 and create a dataset represented as a two-dimensional array (a matrix of objects-features), where each part of the signal of length 1000 is in a separate row.

In [ ]:
def audio_to_matrix(np_array_data, sample_len=1000): # разбиваем аудио на матрицу с шагом sample_len
    return np.array([np_array_data[i:i+sample_len] for i in range(0, len(np_array_data), sample_len)])

#### 1.2. (3 points)

Write a function that translates your `matrix` back into an audio signal. In other words, the function unwraps data from a matrix of size `(number of objects, 1000)` into a vector of length `(number of objects * 1000)`.

Run the function and check that everything works correctly by playing the “restored” signal in your notebook - this signal should be exactly the same as the original one (because, in fact, it is).

In [ ]:
sample_len = 1000
maxtix_audio = audio_to_matrix(mono_sound_to_cut, sample_len)
maxtix_audio.shape # проверим, что размерность матрицы соответствует ожидаемой


In [ ]:
def matrix_to_audio(matrix): # восстанавливаем аудио из матрицы
    return matrix.flatten()

check_audio = matrix_to_audio(maxtix_audio)
check_audio.shape # проверим, что размерность восстановленного аудио соответствует оригиналу


In [ ]:

Audio(check_audio, rate = samplerate) # послушаем, как сказано в задании, что восстановлено верно

In [ ]:
# и не доверяем своим ушам, проверим, что восстановленное аудио соответствует оригиналу
res = np.equal(mono_sound_to_cut, check_audio).all().item()

widgets.Valid(
    value=res,
    description=('They are equal' if res else 'They are not equal'),
)

### 2





#### 2.1. (3 points)

Perform a PCA transformation of our matrix and get the data compressed into a lower dimensional space.

_Hint. At this stage, we have our "dataset" with 1000 "features" and we want to reduce the number of "features" using the PCA method. You are free to choose the number of components, but initially, it is advisable not to choose too small a number. This way, if the result is poor, it will be easier to determine whether the number of components was insufficient or if there was an error somewhere else :)_

    

In [ ]:
features_num = 900

In [ ]:
maxtix_audio = audio_to_matrix(mono_sound_to_cut, sample_len)

def matrix_audio_updater(value):
    global pca
    global pca_maxtix_audio
    global audio_2D_restored
    pca = PCA(n_components=value)
    pca.fit(maxtix_audio)
    pca_maxtix_audio = pca.transform(maxtix_audio)
    audio_2D_restored = pca.inverse_transform(pca_maxtix_audio)
    print("pca_maxtix_audio shape: ", pca_maxtix_audio.shape)
    print("audio_2D_restored shape: ", audio_2D_restored.shape)

interact(matrix_audio_updater, value=(1, sample_len,1))


#### 2.2. (4 points)

Visualize our "objects" in a clear form on a plane. Draw conclusions from their appearance.

_Hint. To do this, you need to apply PCA in some special way, which we also discussed in the seminar._

In [ ]:


def plot_audio_columns(index): # не более чем дань образцу в конспектах
    plt.figure(figsize=(20,5))
    plt.scatter(maxtix_audio[:, index], maxtix_audio[:, index+1], alpha=0.7, color='blue')
    plt.scatter(audio_2D_restored[:, index], audio_2D_restored[:, index+1], color='red', alpha=0.7)
    plt.show()
    # plt.pause(0.25)

interact(plot_audio_columns, index=(0, features_num-1, 2))



In [ ]:
def plot_audio_rows(index): # Если при широком разбросе синих точек красные выдают линию, значит восстановить аудио уже не особо получилось
    plt.figure(figsize=(20,5))
    plt.scatter(range(sample_len), maxtix_audio[index], alpha=0.7, color='blue')
    plt.scatter(range(sample_len), audio_2D_restored[index], color='red', alpha=0.7)
    plt.show()
    # plt.pause(0.25)

interact(plot_audio_rows, index=(0, features_num-1, 1))



In [ ]:
Audio(matrix_to_audio(audio_2D_restored), rate = samplerate) # послушаем, что получилось
# в целом, при 95 компонентах вполне качество восстановления хорошее

#### 2.3. (4 points)

Create a scatter plot of the dataset in space using color. Draw conclusions from this graph. What is depicted overall?

_Hint. That's what we did in the seminar too. This is another special application of PCA._

In [ ]:
def audio_pca(audio_matrix, n_components=20, visualization=True):

    pca = PCA(n_components=n_components)
    Y = pca.fit_transform(audio_matrix)
    X_hat = pca.inverse_transform(Y)

    if visualization:
        pca_vis = PCA(n_components=3) # прям как в конспектах
        Y_vis = pca_vis.fit_transform(audio_matrix)
        plt.figure(figsize=(15, 10))
        plt.scatter(Y_vis[:, 0], Y_vis[:, 1], c=Y_vis[:, 2], alpha=0.5)
        plt.xlabel('Projection onto the first principal component')
        plt.ylabel('Projection onto the second principal component')
        plt.title('Projection onto the first three components (third - color)')
        plt.colorbar()
        plt.show()

    return X_hat

compressed_audio_matrix = audio_pca(maxtix_audio, n_components=3, visualization=True)

for num_components in [900, 500, 100, 75, 60, 50, 40, 30, 20, 10, 5, 3]:
    compressed_audio_matrix = audio_pca(maxtix_audio, n_components=num_components, visualization=False)
    compressed_audio = matrix_to_audio(compressed_audio_matrix)
    print(f"Number of components: {num_components}")
    display(Audio(compressed_audio, rate = samplerate))




Отдельная услада для ушей - слышать, что при сжатии до 3 компонент мы слышим разве что "басы" (в данном случае это пианоно)

А вот тонкие звуки скрипки и флейты мы не слышим и при сжатии до 10 компонент.

Что значит скрипка для ценителей искусства, которым нужно как можно меньше сжатия, а вот современные меломаны, которые слушают музыку в машине, могут и не заметить разницы между 3 и 10 компонентами.

Дальше приводится немного дополнительной визуализации.

In [ ]:
from matplotlib.patches import Ellipse
from ipywidgets import IntSlider


In [ ]:
def plot_pca_interactive(X):
    def update_plot(n_components):

        pca = PCA(n_components=n_components)
        pca.fit(X)
        X_pca = pca.transform(X)

        # очистим с прошлого раза
        plt.clf()
        plt.figure(figsize=(8, 6))
        plt.scatter(X[:, 0], X[:, 1], alpha=0.7, label='Original Data')
        
        for i in range(n_components):
            # Get eigenvector * eigenvalue (for scaling)
            vector = pca.components_[i] * np.sqrt(pca.explained_variance_[i])
            # Plot eigenvector as an arrow
            plt.arrow(pca.mean_[0], pca.mean_[1], vector[0], vector[1],
                      head_width=0.2, head_length=0.3, color=f'C{i}', label=f'PC{i+1}')

            # Calculate ellipse parameters for visualization
            eigenvalue = pca.explained_variance_[i]
            vector = pca.components_[i]
            angle = np.arctan2(vector[1], vector[0])
            angle = np.degrees(angle) 

            # Handle the last ellipse separately to avoid the IndexError
            if i < n_components - 1:
                ellipse = Ellipse(pca.mean_, width=np.sqrt(eigenvalue) * 2, height=np.sqrt(pca.explained_variance_[i+1]) * 2,
                                  angle=angle, edgecolor=f'C{i}', facecolor='none', linewidth=2, alpha=0.7)
            else:
                # For the last ellipse, use the eigenvalue of the last component for both width and height
                ellipse = Ellipse(pca.mean_, width=np.sqrt(eigenvalue) * 2, height=np.sqrt(eigenvalue) * 2,
                                  angle=angle, edgecolor=f'C{i}', facecolor='none', linewidth=2, alpha=0.7)
            plt.gca().add_patch(ellipse)
        
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.title('Interactive PCA')
        plt.legend()
        plt.grid(True)
        plt.show()

    interact(update_plot, n_components=IntSlider(min=1, max=20, step=1, value=2, description='Components:'))
    
plot_pca_interactive(maxtix_audio)

### 3

We need to deal with the actual "compression" of the sound and verify the correctness of our actions.

#### 3.1. (4 points)

Perform inverse PCA transformation of the compressed data and get a “matrix” with the compressed sound. What will be the size of the resulting matrix?

In [ ]:
# Хоть мы это делали ранее при визуализации, но проделаем это наглядно

maxtix_audio = audio_to_matrix(mono_sound_to_cut, sample_len)
print ("maxtix_audio shape: ", maxtix_audio.shape)
pca = PCA(n_components=50)
pca.fit(maxtix_audio)
pca_maxtix_audio = pca.transform(maxtix_audio)
print ("pca_maxtix_audio shape: ", pca_maxtix_audio.shape)
audio_2D_restored = pca.inverse_transform(pca_maxtix_audio)
print ("audio_2D_restored shape: ", audio_2D_restored.shape)

# Ну вот как мы видим, размерность восстановленного аудио совпадает с оригиналом
# То есть мы прошли через узкое горлышко шириной в 50 компонент и восстановили аудио обратно в 1000 сегментов, частота звука осталась прежней.

#### 3.2. (4 points)

Convert the “matrix” obtained in the previous step into a playable signal (our “compressed” mono sound). Play the result `(Audio(YOUR_RESULT, rate = samplerate)`.

Does the resulting sound resemble the original?

In [ ]:
restored_audio = matrix_to_audio(audio_2D_restored)

Audio(restored_audio, rate = samplerate) # послушаем, что получилось

# Does the resulting sound resemble the original? - Ответ зависит от количества компонент, которые мы использовали для сжатия
# В данном случае, для 100 компонент, качество восстановления довольно хорошее
# Для 10 - уже очень печальное

#### 3.3. (8 points)

Conduct a study of the dependence of sound quality on the number of components.

Determine by ear the minimum number of components at which the sound is almost indistinguishable from the original.

Add two variants of the soundtrack to the cells: the original and the one you have chosen. Specify the number of components you have chosen.

_Hint. Try filtering the signal with the `gaussian_filter1d` function from `scipy.ndimage`. This will help remove the unpleasant shot noise when the compression is severe. You'll see what it's all about when you try it in practice._

_Sample code for filtering: `Audio(gaussian_filter1d(mono_sound_compressed, 2), rate = samplerate)`_

In [ ]:
# определение используемой функции лежит внизу, не забудьте ее запустить сначала
# перенёс туда, так как только в последнем задании сказано, обернуть всё в функцию

print ("Original audio: ")
display(Audio(mono_sound_to_cut, rate = samplerate))
print ("Compressed audio: ")
com_number = 70
display(Audio(get_compressed_audio(mono_sound_to_cut, com_number, 1000, True), rate = samplerate))
print("Compession rate: ", 1000 / com_number, " times")

# widgets.HBox(audio_widgets)


#### 3.4. (6 points)
Argumentatively answer the following questions:
1. Q1
    1. Is the number of components you have chosen too many or too few?
    2. How can you determine this?
2. Q2
    1. To what extent can the sound be compressed in this way?
    2. What memory optimization did you achieve in the process?
3. If we are given a different soundtrack, we would need to do all the same things to compress the sound. How do we automatically select the number of components and is it possible?




#### Ответы на вопросы

1. Q1
    1. Слишком мало или слишком много компонент для чего? Для достижения какой цели? В соответствии с условием задачи я выбрал столько компонент, сколько необходимо и достаточно чтобы звук был похож на оригинал. Если бы я выбрал меньше компонент, то звук был бы слишком сжат и не похож на оригинал. Если бы я выбрал больше компонент, то звук был бы похож на оригинал, но сжатие было бы неэффективным.
    2. Как и было сказано в задании - по звуку. В зависимости от конкретного устройства воспроизведения, будь то колонки, наушники-затычки или студийные накладные наушники, а также в зависимости от конкретного человека оценки качества сжатого аудио будут различны. Поэтому в любом случае, при пыпытке достичь наилучшей возможной степени сжатия "без потерь", результат будет субъективен.
2. Q2
    1. В зависимости от желаемой степени качества - от 10 до 20, в некоторых случаях до 30 раз от изначального размера, если нас будет интересовать только что-то похожее на азбуку морзе.
    2. Конкретно при выборе сжатия до 70 компонент при длинне сэмпла 1000, мы сжимаем звук в 14 раз. То есть сэкономили ~93% памяти. Что скорее говорит о низком качестве моего устройства воспроизведения и моего слуха, чем о качестве сжатия.
3. Наилучшим способом определения количества компонент является метод проб и ошибок. Но если у нас есть много данных, то можно использовать методы кросс-валидации и поиска оптимального числа компонент по критериям качества. Например, поиск минимального числа компонент, при котором качество сжатия не ухудшается. Для этого нужно будет ввести некоторый критерий качества. Чем лучше будет критерий, чем больше он будет отражать качество именно то, которое воспринимает человек, тем лучше будет результат.
Также, ввиду того, что было замечено, что при уменьшении числа компонент в первую очередь теряются высокие частоты, то можно использовать методы анализа спектра звука для определения оптимального числа компонент. Например, если в звуке преобладают высокие частоты, то можно сделать вывод, что нужно больше компонент. Если же в звуке преобладают низкие частоты, то можно сделать вывод, что нужно меньше компонент. Конкретные численные значения можно вычислять эвристиками на основе анализа спектра звука.

### 4

#### Extra Research. (15 points).
  
  - Wrap the resulting audio compression code in one or more functions.

  - Investigate how the compression ratio - the ratio of the size of the parts into which the signal was divided in task 1.1 to the size of the space into which you compressed the data using PCA - affects the sound, according to your subjective feelings. Starting from what compression level is the loss of audio track quality noticeable? (Both with and without `gaussian_filter1d` filtering).

  - What does the compression ratio mean for PCA? For a large audio recording (3 min, for example), would we want to break it into more, less, or the same number of segments as the proposed audio recording? Why?

  - Is there any way to automatically select the compression ratio? What is it responsible for in our task? How does the compression ratio affect the sound? Why does it affect the sound in this way?


#### Ответы

  - Wrap the resulting audio compression code in one or more functions.

In [ ]:
# так как в конце задания требуется сравнить качество восстановленного аудио с оригиналом, напишем функцию для этого
def get_compressed_audio(mono_sound, n_components, sample_len, use_gaussian_filter):
    maxtix_audio = audio_to_matrix(mono_sound, sample_len)
    pca = PCA(n_components=n_components)
    pca.fit(maxtix_audio)
    pca_maxtix_audio = pca.transform(maxtix_audio)
    audio_2D_restored = pca.inverse_transform(pca_maxtix_audio)
    if use_gaussian_filter:
        audio_2D_restored = gaussian_filter1d(audio_2D_restored, 2)
    return matrix_to_audio(audio_2D_restored)

# СКО может быть и не лучшим вариантом, но идеала в данном случае нет
def score_compressed_audio(mono_sound, compressed_audio):
    sm = np.sum((mono_sound - compressed_audio)**2)
    return sm / len(mono_sound)

def test_by(sound, n_comps, sample_len, use_gaussian_filter):
    boxes = []
    for n in n_comps:
        box = widgets.Output()
        a = get_compressed_audio(sound, n, sample_len, use_gaussian_filter)
        s = score_compressed_audio(sound, a)
        cr = sample_len / n
        s = '{0:.2f}'.format(s / cr)
        with box:
            display(widgets.Label(str(n) + ' cmps, ' + ('with' if use_gaussian_filter else 'no') + ' filter' + ' Score: ' + s + 
                                  '; '+'{0:.2f}'.format(cr)  + 'x; ' + str(sample_len) + ' s_len'),
                    Audio(a, rate = samplerate))
        boxes.append(box)
    return HBox(boxes)


  - Investigate how the compression ratio - the ratio of the size of the parts into which the signal was divided in task 1.1 to the size of the space into which you compressed the data using PCA - affects the sound, according to your subjective feelings. Starting from what compression level is the loss of audio track quality noticeable? (Both with and without `gaussian_filter1d` filtering).

In [ ]:
n_comps = [100, 70, 50, 30, 10, 3]

v_box = [test_by(mono_sound_to_cut, n_comps, 10000, False),
         test_by(mono_sound_to_cut, n_comps, 10000, True),
         test_by(mono_sound_to_cut, n_comps, 1000, False),
         test_by(mono_sound_to_cut, n_comps, 1000, True),
         test_by(mono_sound_to_cut, n_comps, 500, False),
         test_by(mono_sound_to_cut, n_comps, 500, True),
         test_by(mono_sound_to_cut, n_comps, 100, False),
         test_by(mono_sound_to_cut, n_comps, 100, True),]


display(VBox(v_box))



  - What does the compression ratio mean for PCA? For a large audio recording (3 min, for example), would we want to break it into more, less, or the same number of segments as the proposed audio recording? Why?

What does the compression ratio mean for PCA? - отношение между размером частей, на которые мы разбили сигнал в задаче 1.1, к размеру пространства, в которое вы сжали данные с помощью PCA.
For a large audio recording (3 min, for example), would we want to break it into more, less, or the same number of segments as the proposed audio recording? Why? - как видно из набора аудиосамплов выше увеличение или уменьшение размера сэмпла может солидно ухудшить качество результата. При значительном уменьшении мы захрепим, при значительном увеличении звук "поплывёт", как если бы мы одновременно слышали несколько частей одной и той же мелодии.

  - Is there any way to automatically select the compression ratio? What is it responsible for in our task? How does the compression ratio affect the sound? Why does it affect the sound in this way?

Большую степень сжатия можно применять к аудио, которое содержит меньше высоких частот. Достаточно определить спектрограмму аудио и посмотреть (в кавычках посмотреть, для атоматизации эвристически определить пороги допустимости присутствия для использования того или иного количества компонент) на распределение частот. Если в аудио преобладают низкие частоты, то можно использовать меньше компонент. Если же в аудио преобладают высокие частоты, то придётся использовать больше компонент.

Быстрее всего увеличение степени сжатия проявляется в исчезновении высоких частот. Позже, при дальнейшем увеличении степени сжатия уже будут слышны "хрипы".

Это то, как лично я услышал результаты сжатия. Возможно, у кого-то будет другое мнение.

Если углубляться в суть метода PCA, то более точно можно сформулировать, что при уменьшении числа компонент должны исчезать "детали", а не высокие частоты. То есть алгоритму PCA в целом всё равно какие закономерности находить - те, которые формируют высокие частоты или низкие. Тем не менее, слышно так, как это слышно. В целом уменьшение конкретных частот обычно характерно для сжатия, которое можно найти в, скажем MP3.

Уточняя таким образом предыдущий ответ про автоматизацию, скорее нужно смотереть в спектрограмме не на присутствие высоких частот, а на вид спектрограммы в целом. Скажем, если в спектрограмме более менее одинаково распределены частоты, то это скорее приведёт к тому, что нужно будет использовать больше компонент, так как звук по всей видимости довольно разннобразный и сложный, а если спектрограмма простая, с конкретным пиком, то можно использовать меньше компонент.

Тем не менее, такие выводы стоит делать после изучения большого количества данных, например построить сами спектрограммы, сделать это для разного количества компонент, для разных аудио, посмотреть как влияет на спектрограмму в принципе сжатие и так далее.